In [12]:
# pip install -U transformers
# pip install -U jinja2
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import set_seed

In [ ]:
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [37]:
def chat_prompt(sentence, tok=tokenizer):
    return tok.apply_chat_template(
        [   {"role": "system", "content": "Please respond to the following message as naturally as possible."},
            {"role": "user", "content": sentence}],
        tokenize=False,
        add_generation_prompt=True
    )

In [ ]:
context = "Ellie said, 'The nurse adopted a rescue dog,' and Marco replied, "

In [ ]:
encoded = tokenizer(chat_prompt(context), return_tensors="pt").to(device)
print(encoded)

{'input_ids': tensor([[    1,  9690,   198, 10180,  3535,   288,   260,  1695,  3714,   347,
          6099,   347,  1636,    30,     2,   198,     1,  4093,   198, 36602,
           519,  1137,    28,   637,   504, 12996,  6582,   253, 13650,  2767,
          7904,   284, 33205, 12028,    28,   216,     2,   198,     1,   520,
          9531,   198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
set_seed(1024)

generations = model.generate(
    **encoded, # needs to be a dictionary format
    num_return_sequences=10, 
    max_length=50,
    do_sample=True, 
    top_p=0.8,
    temperature=0.9,
    stop_strings=["."],
    tokenizer=tokenizer
)

In [ ]:
decoded = tokenizer.batch_decode(generations,skip_special_tokens=True)
output = [s.split(context)[-1].strip('\nassistant\n') for s in decoded]
print(output)

["Yes, that's correct, Ellie", "That's a good start.", 'The nurse adopted a rescue dog.', 'That was a delightful coincidence! The nurse', "Yes, that's correct.", "Hello! I'm so glad you asked", "That's a nice detail.", 'Ellie: "The nurse adopted ', "A rescue dog! That's a fantastic", '"Ah, a rescue dog.']
